# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm
tqdm.pandas()
from multiprocessing import  Pool
import time

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt
from matplotlib.pyplot import figure

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global data
    x_axis = []
    for i in range(data['sma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))

    return(slope_tick)

def get_slope_l(y_axis):
    global data
    x_axis = []
    for i in range(data['lma_len']):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global data
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df['spread'] = df['Ask'] - df['Bid']
    df = df[['tick', 'spread']]
    
    temp_df = pd.DataFrame()
    tick_avg = []
    spread_avg = []
    tick_sd = []
    
    for i in tqdm(range(0,len(df),number_of_ticks)):
        tick_list = list(df['tick'][i:i+number_of_ticks])
        spread_list = list(df['spread'][i:i+number_of_ticks])
        
        tick_avg.append(np.mean(tick_list))
        spread_avg.append(np.mean(spread_list))
        tick_sd.append(np.std(tick_list))
        
        
    temp_df['tick_avg'] = tick_avg  
    temp_df['spread_avg'] = spread_avg  
    temp_df['tick_sd'] = tick_sd  
    
    return(temp_df)

In [4]:
def before_sma():
    global data    
    data['ssma_list'].append(val)    
    return()

def after_sma():
    global data
    
    data['ssma_list'].popleft()
    data['ssma_list'].append(val)
    data['sema'] = list(pd.DataFrame(list(data['ssma_list'])).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]
    
    if len(data['sema_ready']) < 2:
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = np.nan

    elif len(data['sema_ready']) > 1:
        data['sema_ready'].popleft()
        data['sema_ready'].append(data['sema'])
        data['sema_diff'] = data['sema_ready'][-1] - data['sema_ready'][len(data['sema_ready'])-2]
    
    return()

def before_lma():
    global data    
    data['lsma_list'].append(val)    
    return()

def after_lma():
    global data
    
    data['lsma_list'].popleft()
    data['lsma_list'].append(val)
    data['lema'] = list(pd.DataFrame(list(data['lsma_list'])).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]
    
    if len(data['lema_ready']) < 2:
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = np.nan

    elif len(data['lema_ready']) > 1:
        data['lema_ready'].popleft()
        data['lema_ready'].append(data['lema'])
        data['lema_diff'] = data['lema_ready'][-1] - data['lema_ready'][len(data['lema_ready'])-2]
    
    return()

In [5]:
def roll_sma(ssma_list):
    global data
    sema_val = list(pd.DataFrame(ssma_list).ewm(span=data['sma_len']).mean()[0])[data['sma_len'] - 1]    
    return(sema_val)

def roll_lma(lsma_list):
    global data
    lema_val = list(pd.DataFrame(ssma_list).ewm(span=data['lma_len']).mean()[0])[data['lma_len'] - 1]    
    return(lema_val)

## File paths

In [6]:
year = 2018
source_file_path = f'data/tick_{year}.csv'
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

chunk_file_name = 'chunk_'+file_name
chunk_file_path = os.path.join(path, chunk_file_name)

print(f'source_file_path : {source_file_path}')
print(f'chunk_file_path : {chunk_file_path}')
print(f'target_file_path : {target_file_path}')

source_file_path : data/tick_2018.csv
chunk_file_path : data\chunk_tick_2018.csv
target_file_path : data\tab_tick_2018.csv


## Read data

In [7]:
%%time
df = pd.read_csv(source_file_path, nrows=1000000)
#df = pd.read_csv(source_file_path)
df.head()

Wall time: 1.1 s


,DateTime,Bid,Ask,Volume
0,20180101 22:00:08.661,1.20102,1.20143,2
1,20180101 22:00:08.895,1.20102,1.20148,2
2,20180101 22:00:10.634,1.20102,1.20147,2
3,20180101 22:00:11.223,1.20102,1.20148,2
4,20180101 22:00:29.530,1.20102,1.20145,2


## Data manipulation

In [8]:
%%time
data = {}
data['number_of_ticks'] = 20
df = chunk_ticks(df, data['number_of_ticks'])
df.to_csv(chunk_file_path, index = False)
print(f'Records : {len(df)}')

df = pd.read_csv(chunk_file_path)
df.head()

100%|██████████████████████████████████████████████████████████████████████████| 50000/50000 [00:22<00:00, 2197.44it/s]


Records : 50000
Wall time: 23.3 s


,tick_avg,spread_avg,tick_sd
0,1.200982,0.000382,0.000245
1,1.200657,0.000704,0.000092
2,1.200720,0.000756,0.000032
3,1.200684,0.000365,0.000109
4,1.200634,0.000248,0.000075


In [9]:
%%time

data['pip_diff'] = 0.00001
data['rsi_window'] = 14
data['rs_max'] = 1e6

data['sma_len'] = 20
data['lma_len'] = 50

data['ssma_list'] = collections.deque([])
data['lsma_list'] = collections.deque([])
data['sema_ready'] = collections.deque([])
data['lema_ready'] = collections.deque([])
df['sema'] = ''
df['lema'] = ''
df['sema_diff'] = ''
df['lema_diff'] = ''


# RSI -----------------------------
df['diff'] = df['tick_avg'].diff()
df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])
df['avg_gain'] = df['gain'].rolling(window=data['rsi_window']).mean()
df['avg_loss'] = df['loss'].rolling(window=data['rsi_window']).mean()
df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= data['rs_max'], data['rs_max']) 
df['rsi'] = 100 - (100 / (df['rs'] + 1))

# Simple Moving Averages ------------------
df['ssma'] = df['tick_avg'].rolling(window=data['sma_len']).mean()
df['ssma_diff'] = df['ssma'].diff()
df['lsma'] = df['tick_avg'].rolling(window=data['lma_len']).mean()
df['lsma_diff'] = df['lsma'].diff()
df['sma_diff'] = df['ssma'] - df['lsma']

df['max_tick'] = df['tick_avg'].rolling(window=data['sma_len']).max()
df['min_tick'] = df['tick_avg'].rolling(window=data['sma_len']).min()

df['max_gap'] = df['max_tick'] -  df['tick_avg']
df['min_gap'] = df['min_tick'] - df['tick_avg']

Wall time: 80.1 ms


In [10]:
%%time
# Emas ----------------
df['sema'] = df['tick_avg'].rolling(window=data['sma_len']).progress_apply(roll_sma)
df['lema'] = df['tick_avg'].rolling(window=data['lma_len']).progress_apply(roll_sma)

df['sema_diff'] = df['sema'].diff()
df['lema_diff'] = df['lema'].diff()

49981it [01:21, 614.31it/s]
49951it [01:19, 630.26it/s]

Wall time: 2min 40s


In [11]:
%%time
# Slopes -----------------------------
df['small_sema_slope'] = df['sema'].rolling(window=data['sma_len']).progress_apply(get_slope_s)
df['long_sema_slope'] = df['sema'].rolling(window=data['lma_len']).progress_apply(get_slope_l)

49962it [00:24, 2067.13it/s]
49932it [00:24, 2037.06it/s]

Wall time: 48.7 s


In [12]:
%%time

df['ema_diff'] = df['sema'] - df['lema']

# Direction -------------------------
diff_col = 'sema'
df['direction'] = 'same'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] > data['pip_diff']] = 'increase'
df['direction'].loc[df[diff_col].shift(-1) - df[diff_col] < -data['pip_diff']] = 'decrease'

# Remove NaNs ------------------------
del df['gain']
del df['loss']
df = df.dropna()
df = df.reset_index(drop=True)
print(f'Total records : {len(df)}')

df.tail()

Total records : 49932
Wall time: 79.9 ms


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
49927,1.229165,0.000038,0.000023,1.228933,1.228841,0.000028,-0.000002,0.000060,0.000040,0.000023,...,-4.150000e-06,-0.000005,1.229165,1.228743,0.0,-0.000422,-20.834396,-28.488234,0.000092,increase
49928,1.229189,0.000037,0.000027,1.228960,1.228846,0.000027,0.000005,0.000024,0.000033,0.000023,...,-2.735000e-06,0.000008,1.229189,1.228743,0.0,-0.000445,-18.690715,-25.887918,0.000115,increase
49929,1.229230,0.000033,0.000027,1.228989,1.228870,0.000028,0.000024,0.000041,0.000036,0.000021,...,7.050000e-07,0.000016,1.229230,1.228743,0.0,-0.000487,-13.105964,-22.717218,0.000119,increase
49930,1.229292,0.000036,0.000019,1.229022,1.228877,0.000033,0.000007,0.000062,0.000040,0.000006,...,1.540000e-06,0.000028,1.229292,1.228743,0.0,-0.000549,-3.878040,-18.849391,0.000144,increase
49931,1.229321,0.000043,0.000016,1.229055,1.228888,0.000033,0.000011,0.000029,0.000042,0.000005,...,4.965000e-06,0.000040,1.229321,1.228743,0.0,-0.000578,8.488709,-14.437940,0.000167,same


## Write data to csv

In [13]:
%%time
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

Wall time: 2.81 s


## Print Report

In [14]:
g= df['direction']
print(pd.concat([g.value_counts(), g.value_counts(normalize=True).mul(100)],axis=1, keys=('counts','percentage')))
df.head(5)

          counts  percentage
same       20860   41.776816
increase   14684   29.407995
decrease   14388   28.815189


,tick_avg,spread_avg,tick_sd,sema,lema,sema_diff,lema_diff,diff,avg_gain,avg_loss,...,lsma_diff,sma_diff,max_tick,min_tick,max_gap,min_gap,small_sema_slope,long_sema_slope,ema_diff,direction
0,1.200596,0.000099,0.000008,1.200862,1.201104,-0.000042,0.000057,0.000001,0.000010,0.000063,...,-2.230000e-06,-0.000046,1.201499,1.200459,0.000902,-0.000138,-72.932497,49.227937,-0.000242,decrease
1,1.200617,0.000073,0.000033,1.200826,1.201157,-0.000036,0.000053,0.000021,0.000012,0.000061,...,-1.970000e-06,-0.000087,1.201499,1.200459,0.000882,-0.000159,-74.594112,45.647158,-0.000331,decrease
2,1.200653,0.000052,0.000022,1.200797,1.201201,-0.000029,0.000044,0.000036,0.000014,0.000056,...,-1.215000e-06,-0.000127,1.201499,1.200459,0.000846,-0.000194,-75.714173,41.217225,-0.000404,decrease
3,1.200679,0.000039,0.000008,1.200774,1.201242,-0.000023,0.000040,0.000026,0.000016,0.000047,...,-7.800000e-07,-0.000167,1.201499,1.200459,0.000820,-0.000220,-76.420008,35.830316,-0.000468,decrease
4,1.200687,0.000039,0.000025,1.200754,1.201279,-0.000020,0.000038,0.000008,0.000017,0.000044,...,-6.750000e-07,-0.000207,1.201461,1.200459,0.000773,-0.000229,-76.810693,29.332969,-0.000526,decrease
